In [100]:
import pandas as pd

In [101]:
df = pd.read_csv('expenses.csv')
people = ['ben','sherina','keiton','chris','hyeok','michael','alex']

df = df.melt(id_vars=[col for col in df.columns if col not in people], var_name='owed_by', value_name='portion')
df.head(3)

,Item,description,amount,paid_by,total,owed_by,portion
0,BNB,NaN,1.00,ben,1,ben,421.30
1,CAR,NaN,1.00,keiton,1,ben,284.51
2,walmart,slippers+tax,9.65,alex,4,ben,0.00


In [102]:
df['paid_by'] = df['paid_by'].apply(lambda row: row if row!='sherina' else 'ben')

In [103]:
df['owed_amount'] = df['amount']*df['portion']/df['total']
df = df[df['owed_amount']!=0]
df = df[df['paid_by']!=df['owed_by']]
df.reset_index(drop = True, inplace=True)
print(len(df), 'transactions to do')
df.head(3)


122 transactions to do


,Item,description,amount,paid_by,total,owed_by,portion,owed_amount
0,CAR,NaN,1.0000,keiton,1,ben,284.51,284.5100
1,walmart,peanut butter,5.0264,alex,1,ben,1.00,5.0264
2,walmart,rest,67.6011,alex,7,ben,1.00,9.6573


In [104]:
for idx, row in df.iterrows():
    print(row['owed_by'], ' owes ', row['paid_by'], ': ', round(row['owed_amount'],2))

ben  owes  keiton :  284.51
ben  owes  alex :  5.03
ben  owes  alex :  9.66
ben  owes  alex :  7.0
ben  owes  keiton :  45.43
ben  owes  chris :  1.83
ben  owes  keiton :  12.48
ben  owes  keiton :  6.96
ben  owes  keiton :  2.65
sherina  owes  ben :  421.3
sherina  owes  keiton :  284.51
sherina  owes  alex :  3.2
sherina  owes  alex :  9.66
sherina  owes  ben :  5.55
sherina  owes  ben :  8.0
sherina  owes  ben :  39.51
sherina  owes  ben :  8.47
sherina  owes  ben :  2.17
sherina  owes  alex :  7.0
sherina  owes  keiton :  45.43
sherina  owes  ben :  7.75
sherina  owes  ben :  1.0
sherina  owes  chris :  1.83
sherina  owes  keiton :  12.48
sherina  owes  ben :  9.86
sherina  owes  ben :  1.9
sherina  owes  ben :  2.17
sherina  owes  ben :  8.86
sherina  owes  ben :  6.67
sherina  owes  keiton :  6.96
sherina  owes  keiton :  2.65
keiton  owes  ben :  421.3
keiton  owes  alex :  2.41
keiton  owes  alex :  8.35
keiton  owes  alex :  3.2
keiton  owes  alex :  9.66
keiton  owes  ben :  

In [105]:
from collections import defaultdict
# Step 1: Calculate net balances for each person
balances = defaultdict(float)
for _, row in df.iterrows():
    balances[row['paid_by']] += row['owed_amount']
    balances[row['owed_by']] -= row['owed_amount']
balances

defaultdict(float,
            {'keiton': 1097.9797323809523,
             'ben': 2110.895752380952,
             'alex': -457.52726761904756,
             'chris': -792.1561342857143,
             'sherina': -896.9269676190476,
             'hyeok': -486.24776761904764,
             'michael': -576.0173476190475})

In [106]:
 # Step 2: Separate into creditors and debtors
creditors = []
debtors = []
for person, balance in balances.items():
    if balance > 0:
        creditors.append((person, balance))
    elif balance < 0:
        debtors.append((person, -balance))
print(creditors)
print(debtors)

[('keiton', 1097.9797323809523), ('ben', 2110.895752380952)]
[('alex', 457.52726761904756), ('chris', 792.1561342857143), ('sherina', 896.9269676190476), ('hyeok', 486.24776761904764), ('michael', 576.0173476190475)]


In [107]:
# Step 3: Minimize transactions
minimized_transactions = []
while creditors and debtors:
    creditor, credit_amount = creditors.pop()
    debtor, debt_amount = debtors.pop()

    payment = min(credit_amount, debt_amount)
    minimized_transactions.append({'paid_by': creditor, 'owed_by': debtor, 'owed_amount': payment})

    if credit_amount > payment:
        creditors.append((creditor, credit_amount - payment))
    if debt_amount > payment:
        debtors.append((debtor, debt_amount - payment))
for transaction in minimized_transactions:
    print(transaction['owed_by'], 'pays', round(transaction['owed_amount'],2), 'to', transaction['paid_by'])

michael pays 576.02 to ben
hyeok pays 486.25 to ben
sherina pays 896.93 to ben
chris pays 151.7 to ben
chris pays 640.45 to keiton
alex pays 457.53 to keiton
